In [1]:
import pandas as pd
import pypsa
from scripts.__helpers import replace_su, average_every_nhours
import numpy as np
import os
import math

In [2]:
from pypsa.descriptors import get_switchable_as_dense as as_dense

In [69]:
def solve_rolling_horizon(m, o, time_aggregation, horizon):
    
    horizon = int(storage_preopt_aggregation.split("h")[0])
    
    o.storage_units.cyclic_state_of_charge=False
    o.stores.e_cyclic=False

    o.stores_t.e_min_pu = (
        (m.stores_t.e/m.stores.e_nom)
        .shift(1)
        .resample(time_aggregation).asfreq()
        .shift(-1)
        .fillna(0)
    ).clip(upper=.999).reindex(o.snapshots).fillna(0)

    o.stores.e_initial = m.stores_t.e.iloc[-1]
    
    for i in range(len(o.snapshots)//horizon):

        start= i* horizon
        end = min((i+1)*horizon, len(o.snapshots))
        
        print("optimizing time period between " + str(o.snapshots[start]) + " and " + str(o.snapshots[end-1]))
        
        snapshots = o.snapshots[start:end]
              
        status, condition = o.optimize(solver_name=solver_name,
                   snapshots = snapshots,
                   assign_all_duals=True,
                   extra_functionality=extra_functionality,
                   linearized_unit_commitment = True,
                   **solver_options
              )
        
        if "infeasible" in condition:
            raise RuntimeError("Solving status 'infeasible'")

        
        
        o.stores.e_initial = o.stores_t.e.loc[snapshots].iloc[-1]

In [4]:
def get_ordc_integral(excess_reserve, mean, std):
    
    return (
        -0.5*excess_reserve 
        + 0.5*(
            (excess_reserve - mean)*math.erf((excess_reserve-mean)/(std*math.sqrt(2)))
            + math.sqrt(2)*std*math.exp(-(excess_reserve-mean)**2/(2*std**2))/math.sqrt(math.pi)
        )
    )

In [5]:
def determine_cutoff(zone):
    cost_function = pd.Series([get_ordc_integral(i, **ordc_parameters.loc[zone]) for i in range(0,40000,10)], 
                              index = range(0,40000,10))

    return cost_function.diff()[cost_function.diff()/pd.Series(cost_function.index).diff().median()<=-0.00001].index[-1]

In [6]:
def get_break_points(n_ints, zone, cut_off):
    
    break_points = pd.DataFrame(index=range(n_ints+1))
    break_points["cap_break_points"] = list(np.arange(0, cut_off[zone], cut_off[zone]/n_ints)) + [2*cut_off[zone]]
    break_points["welfare_break_points"] = [get_ordc_integral(i, **ordc_parameters.loc[zone]) for i in break_points.cap_break_points]
    break_points.set_index(pd.Series(zone,index=break_points.index), inplace=True,append=True)
    break_points = break_points.reorder_levels([1,0])
    
    return break_points

In [7]:
def get_linear_function(p, q):
    m = (p[1] - q[1])/(p[0] - q[0])
    b = (p[0]*q[1] - p[1]*q[0])/(p[0] - q[0])
    return m, b

In [8]:
def get_line_parameters(ordc_parameters, n_ints):
    cut_off = pd.Series(index=ordc_parameters.index, data=[determine_cutoff(i) for i in ordc_parameters.index])

    break_points = pd.DataFrame()
    for zone in ordc_parameters.index:

        break_points = pd.concat([break_points, get_break_points(n_ints, zone, cut_off)])

    line_parameters = pd.DataFrame()
    for zone in ordc_parameters.index:
        j = 0    
        for i in break_points.loc[zone].index[:-1]:
            m, b = get_linear_function(break_points.loc[zone, i], break_points.loc[zone, i+1])
            line_parameters = pd.concat([line_parameters, pd.Series(index=["a", "b"], data=[m, b], name=(zone, j))],axis=1)
            j+=1

    return line_parameters.T.set_index((i for i in line_parameters.columns))

In [9]:
def balancing_market(n, snapshots):
    
    m = n.model

    reserve_gens = n.generators.loc[n.generators.bus.map(n.buses.carrier) == "electricity"]
    reserve_gens = reserve_gens.query("carrier in @reserve_participation_generators")

    coord_reserve_gens = reserve_gens.index
    coord_reserve_gens.name = "Generator-fix"

    gen_r = m.add_variables(lower=0, name="Generator-r", coords = [snapshots, coord_reserve_gens])

    lhs = m.constraints["Generator-fix-p-upper"].lhs
    rhs = m.constraints["Generator-fix-p-upper"].rhs
    m.remove_constraints("Generator-fix-p-upper")
    m.add_constraints(lhs + gen_r <= rhs, name= "Generator-fix-p-upper")

    committable = reserve_gens.query("committable == True").copy()

    committable_grouper = pd.Series(committable.index, committable.index)
    committable_grouper.name="Generator-com"

    lhs = m.constraints["Generator-com-p-upper"].lhs
    rhs = m.constraints["Generator-com-p-upper"].rhs

    lhs += gen_r.loc[:, committable.index].groupby(committable_grouper).sum()
    m.remove_constraints("Generator-com-p-upper")
    m.add_constraints( lhs <= rhs, name="Generator-com-p-upper")

    gen_status = m["Generator-status"]

    reserve_gen_status = gen_status.loc[:, committable.index]

    m.add_constraints(
        gen_r.loc[:, committable.index].groupby(committable_grouper).sum()
        <= (
            reserve_gen_status*
            (
                committable.groupby(committable_grouper).ramp_limit_up.sum()
                .multiply(delivery_time_reserves)
                .multiply(committable.groupby(committable_grouper).p_nom.sum())     
            )
        ),
        name = "Generator-com-ramp_upper"
    )
    


    res_shedding = m.add_variables(
        lower=0, 
        name="Bus-reserve_shedding", 
        coords=[snapshots, reserve_requirements.index]
    )

    storage = n.links.loc[n.links.type == "discharging"]

    storage_link_r = m.add_variables(lower = 0, name="Link-r", coords = m["Link-p"].loc[:, storage.index].coords)

    link_p = m["Link-p"]

    m.add_constraints(
        link_p.loc[:, storage.index] + storage_link_r <= n.links.p_nom.to_xarray(),
        name = "Link-capacity_upper"
    );

    store_e = m["Store-e"]

    store_link_map = n.stores.set_index("bus",append=True).reset_index("Store").reindex(storage.bus0)["Store"]
    store_link_map.index = storage.index
    store_link_map.index.name = "Link"
    store_link_map.name="Store"

    m.add_constraints(
        storage_link_r.groupby(store_link_map.to_xarray()).sum() <= store_e,
        name="Link-reserve_reservoir_constraint"
    )

    gen_grouper = reserve_gens.bus
    gen_grouper.name = "Bus"

    storage_grouper = storage.bus1
    storage_grouper.name = "Bus"

    if ORDC == True:

        excess_r = m.add_variables(
            lower = 0,
            upper= 40,
            name = "Bus-excess_r",
            coords = [snapshots, reserve_requirements.index]
        )

        m.add_constraints(
            gen_r.groupby(gen_grouper.to_xarray()).sum() 
            + storage_link_r.groupby(storage_grouper.to_xarray()).sum()
            + res_shedding
            - excess_r
            >= 0,
            name="Bus-reserve_balance"
        )

        
        a = linear_ordc_approximation["a"]
        b = linear_ordc_approximation["b"]

        a.index.names = ["Bus", "break_point"]
        b.index.names = ["Bus", "break_point"]


        ordc_cost_term = m.add_variables(coords=excess_r.coords, name="Bus-ORDC_cost")
        
        m.add_constraints(
            ordc_cost_term >= (a.to_xarray()*excess_r + b.to_xarray())*VOLL,
            name="Bus-ORDC"
        )

        obj = m.objective
        m.add_objective(obj + res_shedding.sum()*reserve_scarcity_price + ordc_cost_term.sum(), overwrite=True) 

    else:

        m.add_constraints(
            gen_r.groupby(gen_grouper.to_xarray()).sum() 
            + storage_link_r.groupby(storage_grouper.to_xarray()).sum()
            + res_shedding
            >= reserve_requirements.reindex(n.buses.query("carrier == 'electricity'").index).dropna().to_xarray(),
            name="Bus-reserve_balance"
        )

        obj = m.objective

        m.add_objective(obj + res_shedding.sum()*reserve_scarcity_price, overwrite=True)

In [10]:
def flow_based_market_coupling(n, snapshots)   :
    
    m = n.model
    
    ac_core = n.links.query("(carrier == 'AC') & (bus0 in @ptdf_core.columns) & (bus1 in @ptdf_core.columns) ")

    ac_bus0 = ac_core.bus0.copy()
    ac_bus0.name = "bus"

    ac_bus1 = ac_core.bus1.copy()
    ac_bus1.name = "bus"

    net_position_core = m.add_variables(coords=[snapshots, ptdf_core.columns], name="Bus-net_position")

    m.add_constraints(
        net_position_core  == (
            m["Link-p"].loc[:, ac_core.index].groupby(ac_bus0.to_xarray()).sum()
            - m["Link-p"].loc[:, ac_core.index].groupby(ac_bus1.to_xarray()).sum()
        ),
        name="Bus-net_position_zones"
    );

    net_position_alegro = m.add_variables(
        coords=[
            snapshots, 
            pd.Index(["Allegro"], name="Link")
        ],
        name="Net_position_alegro"
    )

    m.add_constraints(
        net_position_alegro == m["Link-p"].loc[:, "BE00-DE00-DC"] - m["Link-p"].loc[:, "DE00-BE00-DC"],
        name="Bus-alegro"
    );

    ahc_flows = n.links[
        (
            (n.links.bus0.isin(ptdf_core.columns) & ~n.links.bus1.isin(ptdf_core.columns))
            | (n.links.bus1.isin(ptdf_core.columns) & ~n.links.bus0.isin(ptdf_core.columns))
        ) & n.links.carrier.isin(['AC', 'DC'])

    ]

    ahc0 = pd.Series(ahc_flows.index.str[:-3], ahc_flows.index)
    ahc0 = ahc0[ahc0.isin(ptdf_ahc.columns)]
    ahc1 = pd.Series([i.split("-")[1] + "-" + i.split("-")[0] for i in ahc_flows.index], ahc_flows.index)
    ahc1 = ahc1[ahc1.isin(ptdf_ahc.columns)]
    ahc0.name = "Flow"
    ahc1.name = "Flow"

    external_flow = m.add_variables(
        coords = [snapshots, ptdf_ahc.columns],
        name="Link-external_flow"
    );

    m.add_constraints(
        external_flow == m["Link-p"].loc[:,ahc0.index].groupby(ahc0.to_xarray()).sum() - m["Link-p"].loc[:,ahc1.index].groupby(ahc1.to_xarray()).sum(),
        name="Link-external_flows"
    );

    m.add_constraints(
        (
            (ptdf_ahc.loc[snapshots].stack().to_xarray()*external_flow).sum("Flow")  
            + (ptdf_core.loc[snapshots].stack().to_xarray()*net_position_core).sum("bus")
            + (net_position_alegro*ptdf_dc.loc[snapshots].ALEGRO.to_xarray()).sum("Link")
        ) <= ram.loc[snapshots].to_xarray(),
        name="Link-RAM_constraint"
    )

In [11]:
def build_flow_based_data(excel_file):

    ptdf_core = pd.read_excel(excel_file, "PTDF", index_col=[0,1,2], header=[0,1])["PTDF_SZ"]
    ptdf_ahc = pd.read_excel(excel_file, "PTDF", index_col=[0,1,2], header=[0,1])["PTDF*_AHC,SZ"]
    ptdf_dc = pd.read_excel(excel_file, "PTDF", index_col=[0,1,2], header=[0,1])["PTDF_EvFB"]
    
    domain_assign = pd.read_excel(excel_file, "FB Domain Assignment", index_col=[0]).iloc[:, 4:]
    domain_assign.columns = [int(i.split("_")[1]) for i in domain_assign.columns]
    domain_assign = domain_assign[climate_year]

    ptdf_core = ptdf_core.reset_index(2,drop=True).unstack(1).reindex(domain_assign.values)
    ptdf_core.index = n.snapshots
    ptdf_core = ptdf_core.stack(1)

    ptdf_ahc = ptdf_ahc.reset_index(2,drop=True).unstack(1).reindex(domain_assign.values)
    ptdf_ahc.index = n.snapshots
    ptdf_ahc = ptdf_ahc.stack(1)

    ptdf_dc = ptdf_dc.reset_index(2, drop=True).unstack(1).reindex(domain_assign.values)
    ptdf_dc.index = n.snapshots
    ptdf_dc = ptdf_dc.stack(1)


    ram_sheets = [i for i in excel_file.sheet_names if "RAM" in i]

    base_years = pd.Series([int(i.split("_")[1]) for i in ram_sheets], ram_sheets) - target_year

    ram_sheet = base_years[base_years <=0].idxmax()
    ram = pd.read_excel(excel_file, ram_sheet, index_col=[0], skiprows=3)
    ram.columns = ram.columns.astype(int)
    ram = ram.T.reindex(domain_assign.values)
    ram.index = n.snapshots
    ram = ram.stack()

    ptdf_core.index.names = ["snapshot", "CNEC"]
    ptdf_core.columns.name = "bus"
    ptdf_ahc.index.names = ["snapshot", "CNEC"]
    ptdf_ahc.columns.name = "Flow"
    ptdf_dc.index.names = ["snapshot", "CNEC"]
    ram.index.names = ["snapshot", "CNEC"]
    
    ptdf_core = ptdf_core.reindex(
        pd.MultiIndex.from_product(
            [ptdf_core.index.levels[0], ptdf_core.index.levels[1]]
        ),
        fill_value=0
    )
    
    ptdf_ahc = ptdf_ahc.reindex(
        pd.MultiIndex.from_product(
            [ptdf_ahc.index.levels[0], ptdf_ahc.index.levels[1]]
        ),
        fill_value=0
    )
    ptdf_dc = ptdf_dc.reindex(
        pd.MultiIndex.from_product(
            [ptdf_dc.index.levels[0], ptdf_dc.index.levels[1]]
        ), 
        fill_value=0
    )
    
    ptdf_ahc = ptdf_ahc[ptdf_ahc.sum()[ptdf_ahc.sum()!=0].index]
    ptdf_ahc = ptdf_ahc[[i for i in ptdf_ahc.columns if i.split("-")[0][:4] in n.buses.index and i.split("-")[1][:4] in n.buses.index]]
    ptdf_ahc = ptdf_ahc.groupby(ptdf_ahc.columns.str[:9],axis=1).sum()
    
    return ptdf_core, ptdf_ahc, ptdf_dc, ram

In [12]:
def extra_functionality(n, snapshots):
    balancing_market(n, snapshots)
    if FBMC == True:
        flow_based_market_coupling(n, snapshots)

In [13]:
target_year = 2029

In [14]:
climate_year=1990

In [15]:
solved_network = "results/networks/base/cy1992_ty2026.nc"

In [16]:
revenues = "results/revenues/base_cy1992_ty2026.h5"

In [17]:
n = pypsa.Network("resources/networks/base/cy1982_ty2025.nc")

INFO:pypsa.io:Imported network cy1982_ty2025.nc has buses, generators, links, loads, storage_units


In [18]:
years = range(2025, 2034)

In [19]:
reserve_requirements = pd.read_excel("data/pemmdb.xlsx", "Reserve Requirements",index_col=[0,1])[["FCR (MW)", "FRR (MW)"]].sum(axis=1)#.loc[:, target_year]
reserve_requirements = reserve_requirements.unstack(1).reindex(years, axis=1).interpolate(axis=1)[target_year]
reserve_requirements.index.name = "Bus"
reserve_requirements = reserve_requirements[reserve_requirements>0]

In [20]:
solver_name="highs"

In [21]:
flow_based = pd.ExcelFile("data/fb_domains.xlsx")

In [22]:
ptdf_core, ptdf_ahc, ptdf_dc, ram = build_flow_based_data(flow_based)

In [23]:
solver_name="cplex"
solver_options= {
    "threads": 4,
    "lpmethod": 4,
    "solutiontype": 1,
    "barrier.convergetol": 1e-5,
    "feasopt.tolerance": 1e-5,
    "barrier.limits.objrange":1e75,
    "barncolz":150,
}

In [24]:
climate_year = 1996

In [25]:
target_year = 2026

In [26]:
VOLL = 10e3
reserve_scarcity_price = 15e3
FBMC = True

In [27]:
reserve_participation_generators = ['CCGT', 'OCGT', 'oil', 'biomass', 'other', 'lignite', 'nuclear', 'coal']

In [28]:
ORDC = True

In [29]:
for su in n.storage_units.index:
    replace_su(n, su);

In [70]:
storage_preopt_aggregation = "48h"

In [31]:
delivery_time_reserves = 15 # in minutes
delivery_time_reserves = delivery_time_reserves/60

In [32]:
n.madd("Generator", 
       n.buses.query("carrier == 'electricity'").index + " load-shedding", 
       bus = n.buses.query("carrier == 'electricity'").index, 
       p_nom = 1e6, 
       marginal_cost = VOLL
      )

Index(['AL00 load-shedding', 'AT00 load-shedding', 'BA00 load-shedding',
       'BE00 load-shedding', 'BG00 load-shedding', 'CH00 load-shedding',
       'CY00 load-shedding', 'CZ00 load-shedding', 'DE00 load-shedding',
       'DEKF load-shedding', 'DKE1 load-shedding', 'DKKF load-shedding',
       'DKW1 load-shedding', 'EE00 load-shedding', 'ES00 load-shedding',
       'FI00 load-shedding', 'FR00 load-shedding', 'GR00 load-shedding',
       'GR03 load-shedding', 'HR00 load-shedding', 'HU00 load-shedding',
       'IE00 load-shedding', 'ITCA load-shedding', 'ITCN load-shedding',
       'ITCS load-shedding', 'ITN1 load-shedding', 'ITS1 load-shedding',
       'ITSA load-shedding', 'ITSI load-shedding', 'LT00 load-shedding',
       'LUG1 load-shedding', 'LV00 load-shedding', 'ME00 load-shedding',
       'MK00 load-shedding', 'MT00 load-shedding', 'NL00 load-shedding',
       'NOM1 load-shedding', 'NON1 load-shedding', 'NOS0 load-shedding',
       'PL00 load-shedding', 'PLE0 load-shedding', 

In [33]:
n.madd("Generator", 
       n.buses.query("carrier == 'electricity'").index + " gen-shedding", 
       bus = n.buses.query("carrier == 'electricity'").index, 
       p_nom = 1e6, 
       p_min_pu= -1,
       p_max_pu = 0,
       marginal_cost = -500
      )

Index(['AL00 gen-shedding', 'AT00 gen-shedding', 'BA00 gen-shedding',
       'BE00 gen-shedding', 'BG00 gen-shedding', 'CH00 gen-shedding',
       'CY00 gen-shedding', 'CZ00 gen-shedding', 'DE00 gen-shedding',
       'DEKF gen-shedding', 'DKE1 gen-shedding', 'DKKF gen-shedding',
       'DKW1 gen-shedding', 'EE00 gen-shedding', 'ES00 gen-shedding',
       'FI00 gen-shedding', 'FR00 gen-shedding', 'GR00 gen-shedding',
       'GR03 gen-shedding', 'HR00 gen-shedding', 'HU00 gen-shedding',
       'IE00 gen-shedding', 'ITCA gen-shedding', 'ITCN gen-shedding',
       'ITCS gen-shedding', 'ITN1 gen-shedding', 'ITS1 gen-shedding',
       'ITSA gen-shedding', 'ITSI gen-shedding', 'LT00 gen-shedding',
       'LUG1 gen-shedding', 'LV00 gen-shedding', 'ME00 gen-shedding',
       'MK00 gen-shedding', 'MT00 gen-shedding', 'NL00 gen-shedding',
       'NOM1 gen-shedding', 'NON1 gen-shedding', 'NOS0 gen-shedding',
       'PL00 gen-shedding', 'PLE0 gen-shedding', 'PLI0 gen-shedding',
       'PT00 gen-she

In [34]:
ordc_parameters = pd.read_hdf("data/ordc_parameters.h5", "ordc_parameters")

In [35]:
m = average_every_nhours(n, storage_preopt_aggregation)

In [36]:
m.generators["p_min_pu"] = 0

In [37]:
m.stores.e_cyclic = True

In [38]:
import datetime

In [61]:
solver_name="cplex"
solver_options= {
    "threads": 4,
    "lpmethod": 4,
    "solutiontype": 1,
    "barrier.convergetol": 1e-5,
    "feasopt.tolerance": 1e-5,
    "barrier.limits.objrange":1e75,
    "barrier.crossover":1,
    "barrier.display":2,
    "barrier.algorithm":3,
    "barrier.startalg": 2,
    "preprocessing.dual":-1,
    "simplex.pgradient": 3,
    #"barrier.colnonzeros": 50,
}

In [40]:
m.generators.committable = False

In [41]:
start = datetime.datetime.now()
status, condition = m.optimize(
    solver_name=solver_name,
    **solver_options
)

if "infeasible" in condition:
    raise RuntimeError("Solving status 'infeasible'")
    
end = datetime.datetime.now()

print(end- start)

INFO:linopy.model: Solve problem using Cplex solver
INFO:linopy.model:Solver options:
 - threads: 4
 - lpmethod: 4
 - solutiontype: 1
 - barrier.convergetol: 1e-05
 - feasopt.tolerance: 1e-05
 - barrier.limits.objrange: 1e+75
 - barrier.crossover: 1
 - barrier.display: 2
 - barrier.algorithm: 3
 - barrier.startalg: 2
 - preprocessing.dual: -1
 - barrier.colnonzeros: 3869
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 11.55it/s]
INFO:linopy.io: Writing time: 2.58s


Checking license ...


cpxchecklic: /lib64/libcurl.so.4: no version information available (required by cpxchecklic)



License found. [0.03 s]
Version identifier: 22.1.1.0 | 2022-11-26 | 9160aff4d
CPXPARAM_Preprocessing_Dual                      -1
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 4
CPXPARAM_SolutionType                            1
CPXPARAM_Barrier_Algorithm                       3
CPXPARAM_Barrier_ColNonzeros                     3869
CPXPARAM_Barrier_Display                         2
CPXPARAM_Barrier_StartAlg                        2
CPXPARAM_Barrier_Crossover                       1
CPXPARAM_Feasopt_Tolerance                       1.0000000000000001e-05
CPXPARAM_Barrier_ConvergeTol                     1.0000000000000001e-05
CPXPARAM_Barrier_Limits_ObjRange                 9.9999999999999993e+74
Tried aggregator 1 time.
LP Presolve eliminated 552226 rows and 63377 columns.
Aggregator did 15189 substitutions.
Reduced LP has 23630 rows, 158053 columns, and 205239 nonzeros.
Presolve time


Total time on 4 threads = 1.72 sec. (2218.04 ticks)


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 236619 primals, 591045 duals
Objective: 4.76e+10
Solver model: available
Solver message: optimal

/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/linopy/common.py:124: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn("Coordinates across variables not equal. Perform outer join.", UserWarning)
INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down, Link-fix-p-lower, Link-fix-p-upper, Store-fix-e-lower, Store-fix-e-upper, Store-energy_balance were not assigned to the network.


0:00:07.098476


In [43]:
dispatchable = ['CCGT', 'OCGT', 'oil', 'biomass', 'other', 'lignite', 'nuclear', 'coal']
n.generators.loc[n.generators.query("carrier in @dispatchable").index, "committable"] = True
n.generators.shut_down_cost = n.generators.start_up_cost

In [44]:
linear_ordc_approximation = get_line_parameters(ordc_parameters, n_ints = 5)

In [46]:
o = m.copy()

In [47]:
m = o.copy()

In [48]:
if FBMC == True:
    n.links.loc[n.links.bus0.isin(ptdf_core.columns) & n.links.bus1.isin(ptdf_core.columns), "p_nom"] *=5

48

In [71]:
solve_rolling_horizon(m, n, "h", storage_preopt_aggregation)

optimizing time period between 2010-01-01 00:00:00 and 2010-01-02 23:00:00


INFO:linopy.model: Solve problem using Cplex solver
INFO:linopy.model:Solver options:
 - threads: 4
 - lpmethod: 4
 - solutiontype: 1
 - barrier.convergetol: 1e-05
 - feasopt.tolerance: 1e-05
 - barrier.limits.objrange: 1e+75
 - barrier.crossover: 1
 - barrier.display: 2
 - barrier.algorithm: 3
 - barrier.startalg: 2
 - preprocessing.dual: -1
 - simplex.pgradient: 3
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 58.38it/s]
INFO:linopy.io: Writing time: 4.05s


Checking license ...


cpxchecklic: /lib64/libcurl.so.4: no version information available (required by cpxchecklic)



License found. [0.14 s]
Version identifier: 22.1.1.0 | 2022-11-26 | 9160aff4d
CPXPARAM_Simplex_PGradient                       3
CPXPARAM_Preprocessing_Dual                      -1
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 4
CPXPARAM_SolutionType                            1
CPXPARAM_Barrier_Algorithm                       3
CPXPARAM_Barrier_Display                         2
CPXPARAM_Barrier_StartAlg                        2
CPXPARAM_Barrier_Crossover                       1
CPXPARAM_Feasopt_Tolerance                       1.0000000000000001e-05
CPXPARAM_Barrier_ConvergeTol                     1.0000000000000001e-05
CPXPARAM_Barrier_Limits_ObjRange                 9.9999999999999993e+74
Tried aggregator 1 time.
LP Presolve eliminated 161308 rows and 29386 columns.
Aggregator did 5830 substitutions.
Reduced LP has 168915 rows, 96544 columns, and 1174695 nonzeros.
Presolve time = 

...

    33398 DMoves:  Infeasibility  1.95342402e+03  Objective  7.69441528e+08
    31594 DMoves:  Infeasibility  1.95342402e+03  Objective  7.69441528e+08
    29739 DMoves:  Infeasibility  1.93804984e+03  Objective  7.69441528e+08
    26922 DMoves:  Infeasibility  1.93804984e+03  Objective  7.69441528e+08
    24671 DMoves:  Infeasibility  1.93804984e+03  Objective  7.69441528e+08
    22574 DMoves:  Infeasibility  1.93804984e+03  Objective  7.69441528e+08
     9779 DMoves:  Infeasibility  1.89767808e+03  Objective  7.69441528e+08
     7570 DMoves:  Infeasibility  1.84135994e+03  Objective  7.69441528e+08
     5726 DMoves:  Infeasibility  1.83027366e+03  Objective  7.69441528e+08
     3881 DMoves:  Infeasibility  1.67285196e+03  Objective  7.69430448e+08
     1916 DMoves:  Infeasibility  1.67223047e+03  Objective  7.69430448e+08
        0 DMoves:  Infeasibility  5.67440700e+02  Objective  7.69430447e+08
  Dual:  Pushed 26722, exchanged 17020.

Iteration log . . .
Iteration:     1    Objecti

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 131280 primals, 336592 duals
Objective: 7.69e+08
Solver model: available
Solver message: optimal

/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/linopy/common.py:124: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn("Coordinates across variables not equal. Perform outer join.", UserWarning)
/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/linopy/common.py:124: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn("Coordinates across variables not equal. Perform outer join.", UserWarning)
INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Bus-ORDC were not assigned to the network.


optimizing time period between 2010-01-09 00:00:00 and 2010-01-10 23:00:00


INFO:linopy.model: Solve problem using Cplex solver
INFO:linopy.model:Solver options:
 - threads: 4
 - lpmethod: 4
 - solutiontype: 1
 - barrier.convergetol: 1e-05
 - feasopt.tolerance: 1e-05
 - barrier.limits.objrange: 1e+75
 - barrier.crossover: 1
 - barrier.display: 2
 - barrier.algorithm: 3
 - barrier.startalg: 2
 - preprocessing.dual: -1
 - simplex.pgradient: 3
INFO:linopy.io:Writing objective.
Writing constraints.:  97%|████████████████████████████████████████████████████████████████████████████████▎  | 30/31 [00:03<00:00,  9.76it/s]


KeyboardInterrupt: 

In [ ]:
dirname = os.path.dirname(solved_network)

In [ ]:
if not os.path.isdir(dirname):
    os.makedirs(dirname)

In [ ]:
def market_revenue(n):
    
    spot_prices = n.buses_t.marginal_price.reindex(n.generators.bus.values, axis=1)

    spot_prices.columns = n.generators.index

    spot_payments = spot_prices.subtract(
        n.generators.marginal_cost
    ).multiply(n.generators_t.p).sum()

    reserve_prices = n.buses_t.mu_reserve_balance.reindex(n.generators.bus.values, axis=1)

    reserve_prices.columns = n.generators.index

    reserve_payments = reserve_prices.multiply(n.generators_t.r).sum()

    return (spot_payments + reserve_payments)

In [ ]:
if not os.path.exists(os.path.dirname(revenues)):
    os.makedirs(os.path.dirname(revenues))

In [ ]:
market_revenue(n).to_hdf(revenues, "revenues")

In [ ]:
(reserve_payments + spot_payments).div(n.generators.p_nom).sum()

In [ ]:
n.generators_t.p["AL00 CCGT"].iloc[:15*24].plot()

n.buses_t.marginal_price["AL00"].iloc[:15*24].plot()

In [ ]:
gradient_ramps = n.generators_t.status.multiply(
    n.generators.ramp_limit_up.subtract(
        n.generators.ramp_limit_start_up)
).subtract(
    n.generators_t.status.shift(1).multiply(
        n.generators.ramp_limit_start_up
    )
).multiply(n.generators_t.mu_ramp_limit_up).sum()

In [ ]:
(gradient_ramps + n.generators_t.mu_upper.sum())

In [ ]:
gradient_upper = 

In [ ]:
n.generators_t.mu_upper.sum().sum()/1e6

In [ ]:
n.generators_t.mu_up.sum()

In [ ]:
spot_prices = n.buses_t.marginal_price.reindex(n.generators.bus.values, axis=1)

In [ ]:
spot_prices.columns = n.generators.index

In [ ]:
spot_payments = spot_prices.subtract(
    n.generators.marginal_cost
).multiply(n.generators_t.p).sum()

In [ ]:
reserve_prices = n.buses_t.mu_reserve_balance.reindex(n.generators.bus.values, axis=1)

In [ ]:
reserve_prices.columns = n.generators.index

In [ ]:
reserve_payments = reserve_prices.multiply(n.generators_t.r).sum()

In [ ]:
(reserve_payments + spot_payments).div(n.generators.p_nom).sum()

In [ ]:
n_copy = n.copy()

In [ ]:
snapshots = n.snapshots[:24]

In [ ]:
n_copy.set_snapshots(snapshots)

In [ ]:
n_copy.generators.ramp_limit_up = 0.4
n_copy.generators.ramp_limit_down = 0.5
n_copy.generators.ramp_limit_start_up = 0.6
n_copy.generators.ramp_limit_start_down = 0.7


In [ ]:
n_copy.optimize(
    solver_name="highs", 
    linearized_unit_commitment=True,
    extra_functionality=extra_functionality,
    assign_all_duals=True
)

In [ ]:
n_copy.model

In [ ]:
m = n_copy.model

In [ ]:
constraint_list = [i for i in m.constraints if "Generator-com" in i]

In [ ]:
constraint_list[0]

In [ ]:
n_copy.generators.p_nom.multiply(n_copy.generators.p_min_pu)

In [ ]:
n.generators_t.mu_lower.sum()

In [ ]:
25.62/89.04

In [ ]:
n.generators.columns

In [ ]:
m.constraints[constraint_list[13]]

In [ ]:
n.generators_t.mu_up.sum()

In [ ]:
n.generators_t.mu_before.multiply(as_dense(n, "Generator", "p_max_pu")).sum()

In [ ]:
n.generators_t.mu_current.multiply(as_dense(n, "Generator", "p_max_pu")).sum()

In [ ]:
m.constraints["Generator-com-p-before"].lhs.loc[n_copy.snapshots[2], :]

In [ ]:
m.constraints["Generator-com-p-current"]